In [1]:
import sys
import os
import numpy as np

# Add the source directory to the path to import custom modules
sys.path.append(os.path.abspath(os.path.join('..')))

from src import data_processing as dp

# Load the raw data
filepath = '../data/raw/aug_train.csv'
raw_data = dp.load_data(filepath)

# Fix string column lengths to prevent truncation
string_length_map = {
    'enrolled_university': 25,
    'education_level': 25,
    'major_discipline': 30,
    'company_type': 30,
    'last_new_job': 10,
    'gender': 10
}
#raw_data = dp.update_string_lengths(raw_data, string_length_map)

print(f"Raw data loaded with shape: {raw_data.shape}")

Raw data loaded with shape: (19158,)


# Preprocessing

## 1. Loại bỏ các cột không cần thiết

Dựa trên phân tích ở notebook trước, chúng ta sẽ loại bỏ các cột `enrollee_id` và `city`. 
- `enrollee_id` là một mã định danh không có giá trị dự đoán.
- `city` chứa thông tin dư thừa đã được thể hiện trong `city_development_index`. 

In [2]:
print(f"Cột ban đầu: {raw_data.dtype.names}\n")

data_step1 = dp.remove_columns(raw_data, ['enrollee_id', 'city'])

print(f"Cột sau khi loại bỏ: {data_step1.dtype.names}")

Cột ban đầu: ('enrollee_id', 'city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target')

Cột sau khi loại bỏ: ('city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target')


## 2. Xử lý dữ liệu trùng lặp

Chúng ta đã xác định được 49 dòng dữ liệu bị trùng lặp (không tính `enrollee_id`). Các dòng này sẽ được loại bỏ để đảm bảo tính toàn vẹn của dữ liệu.

In [3]:
print(f"Kích thước trước khi xóa trùng lặp: {data_step1.shape}\n")

data_step2 = dp.remove_duplicates(data_step1)

print(f"Kích thước sau khi xóa trùng lặp: {data_step2.shape}")
print(f"Số dòng đã xóa: {data_step1.shape[0] - data_step2.shape[0]}")

Kích thước trước khi xóa trùng lặp: (19158,)

Kích thước sau khi xóa trùng lặp: (19098,)
Số dòng đã xóa: 60


## 3. Xử lý giá trị thiếu cho biến phân loại

Như đã thảo luận, chúng ta sẽ không xóa hay điền các giá trị thiếu bằng mode. Thay vào đó, chúng ta sẽ coi "thiếu" là một hạng mục riêng biệt. Chúng ta sẽ thay thế các giá trị rỗng (`''`) trong các cột phân loại bằng một nhãn mới là `"Missing"`.

In [4]:
# Identify categorical columns with missing values
missing_summary = dp.get_missing_summary(data_step2)
cols_to_fill = [
    col for col, summary in missing_summary.items() 
    if summary['missing_count'] > 0 and data_step2.dtype[col].kind == 'U'
]

print(f"Các cột sẽ được xử lý: {cols_to_fill}\n")

data_step3 = dp.fill_missing_categorical(data_step2, columns=cols_to_fill, fill_value='Missing')

# Verify by checking unique values in some of the processed columns
print("Kiểm tra giá trị duy nhất trong cột 'gender' sau khi xử lý:")
print(np.unique(data_step3['gender']))
print("\nKiểm tra giá trị duy nhất trong cột 'company_type' sau khi xử lý:")
print(np.unique(data_step3['company_type']))

Các cột sẽ được xử lý: ['company_type', 'company_size', 'gender', 'major_discipline', 'education_level', 'last_new_job', 'enrolled_university', 'experience']

Kiểm tra giá trị duy nhất trong cột 'gender' sau khi xử lý:
['Female' 'Male' 'Missing' 'Other']

Kiểm tra giá trị duy nhất trong cột 'company_type' sau khi xử lý:
['Early Stage Startup' 'Funded Startup' 'Missing' 'NGO' 'Other'
 'Public Sector' 'Pvt Ltd']


## 4. Nhóm các giá trị `experience`

Cột `experience` hiện đang chứa nhiều giá trị rời rạc. Để mô hình có thể học tốt hơn, chúng ta sẽ nhóm chúng vào các khoảng kinh nghiệm có ý nghĩa hơn: `<1`, `1-5`, `6-10`, `11-15`, `16-20`, và `>20`. 

In [5]:
print(f"Các giá trị experience ban đầu: {np.unique(data_step2['experience'])}\n")

data_step4 = dp.group_experience(data_step3)

print(f"Các giá trị experience sau khi nhóm: {np.unique(data_step4['experience'])}")

Các giá trị experience ban đầu: ['' '1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '3' '4'
 '5' '6' '7' '8' '9' '<1' '>20']

Các giá trị experience sau khi nhóm: ['' '1-5' '11-15' '16-20' '6-10' '<1' '>20']


## Dữ liệu sau khi tiền xử lý

Đây là tóm tắt của dữ liệu sau khi hoàn thành các bước tiền xử lý cơ bản.

In [6]:
preprocessed_data = data_step4

print("Tóm tắt dữ liệu sau tiền xử lý:")
basic_info = dp.get_basic_info(preprocessed_data)
print(f"- Số dòng: {basic_info['shape'][0]}")
print(f"- Số cột: {basic_info['shape'][1]}")
print(f"- Các cột: {basic_info['columns']}")
print("\n5 dòng đầu tiên:")
print(preprocessed_data[:5])

Tóm tắt dữ liệu sau tiền xử lý:
- Số dòng: 19098
- Số cột: 12
- Các cột: ('city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target')

5 dòng đầu tiên:
[(0.92 , 'Male', 'Has relevent experience', 'no_enrollment', 'Graduate', 'STEM', '>20', 'Missing', 'Missing', '1', 36, 1.)
 (0.776, 'Male', 'No relevent experience', 'no_enrollment', 'Graduate', 'STEM', '11-15', '50-99', 'Pvt Ltd', '>4', 47, 0.)
 (0.624, 'Missing', 'No relevent experience', 'Full time course', 'Graduate', 'STEM', '1-5', 'Missing', 'Missing', 'never', 83, 0.)
 (0.789, 'Missing', 'No relevent experience', 'Missing', 'Graduate', 'Business Degree', '<1', 'Missing', 'Pvt Ltd', 'never', 52, 1.)
 (0.767, 'Male', 'Has relevent experience', 'no_enrollment', 'Masters', 'STEM', '>20', '50-99', 'Funded Startup', '4',  8, 0.)]


In [7]:
categorical_cols = [name for name, dtype in data_step4.dtype.fields.items() if not np.issubdtype(dtype[0], np.number)]

for col in categorical_cols:
    unique_vals = np.unique(data_step4[col])
    if col == 'city':
        # Chỉ in số giá trị vì cột city có quá nhiều giá trị
        print(f"- Cột '{col}' ({len(unique_vals)} giá trị khác)")
        continue
    print(f"- Cột '{col}' ({len(unique_vals)} giá trị khác nhau): {unique_vals}")

- Cột 'gender' (4 giá trị khác nhau): ['Female' 'Male' 'Missing' 'Other']
- Cột 'relevent_experience' (2 giá trị khác nhau): ['Has relevent experience' 'No relevent experience']
- Cột 'enrolled_university' (4 giá trị khác nhau): ['Full time course' 'Missing' 'Part time course' 'no_enrollment']
- Cột 'education_level' (6 giá trị khác nhau): ['Graduate' 'High School' 'Masters' 'Missing' 'Phd' 'Primary School']
- Cột 'major_discipline' (7 giá trị khác nhau): ['Arts' 'Business Degree' 'Humanities' 'Missing' 'No Major' 'Other' 'STEM']
- Cột 'experience' (7 giá trị khác nhau): ['' '1-5' '11-15' '16-20' '6-10' '<1' '>20']
- Cột 'company_size' (9 giá trị khác nhau): ['10/49' '100-500' '1000-4999' '10000+' '50-99' '500-999' '5000-9999'
 '<10' 'Missing']
- Cột 'company_type' (7 giá trị khác nhau): ['Early Stage Startup' 'Funded Startup' 'Missing' 'NGO' 'Other'
 'Public Sector' 'Pvt Ltd']
- Cột 'last_new_job' (7 giá trị khác nhau): ['1' '2' '3' '4' '>4' 'Missing' 'never']


## 4. Xuất dữ liệu đã xử lý

Cuối cùng, chúng ta sẽ lưu dữ liệu đã được làm sạch và tiền xử lý vào một tệp CSV mới để sử dụng trong giai đoạn mô hình hóa.

In [8]:
# Define the output path
output_dir = '../data/processed'
output_filepath = os.path.join(output_dir, 'aug_train_processed.csv')

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Create the header string
header = ','.join(preprocessed_data.dtype.names)

# Save the structured array to a CSV file
# A generic '%s' format works by converting all fields to their string representation.
np.savetxt(output_filepath, preprocessed_data, delimiter=',', fmt='%s', header=header, comments='')

print(f"Data successfully exported to: {output_filepath}")

Data successfully exported to: ../data/processed/aug_train_processed.csv
